In [1]:
%load_ext snakeviz
from tyssue import Sheet, SheetGeometry, config
from tyssue.dynamics import SheetModel
from tyssue.solvers.sheet_vertex_solver import Solver
from tyssue.io.hdf5 import load_datasets


In [2]:
ls ../data/hf5/

before_apoptosis_anchors.hf5  ellipsoid_sheet_init.hf5  small_ellipsoid.hf5
before_apoptosis.hf5          fullerene.hf5             small_hexagonal.hf5
ellipsoid.hf5                 lemon.hf5                 small_organo.hf5
ellipsoid_sheet_5k.hf5        organo.hf5
ellipsoid_sheet.hf5           small_cylindrical.hf5


In [3]:

sheet = Sheet('optim', load_datasets('../data/hf5/before_apoptosis.hf5'),
              config.geometry.cylindrical_sheet())

nd_specs = config.dynamics.quasistatic_sheet_spec()
d_specs = SheetModel.dimensionalize(nd_specs)
sheet.update_specs(d_specs)

In [4]:
%%snakeviz
Solver.find_energy_min(sheet, SheetGeometry, SheetModel)

 
*** Profile stats marshalled to file '/tmp/tmpkdmjmjw9'. 


In [9]:
import tyssue

In [11]:
tyssue.version.full_version

'0.2.2.dev0+2f9600e'

In [12]:
!cp /tmp/tmpkdmjmjw9 ../data/optim/opt_2f9600e.prof

In [13]:
%timeit SheetGeometry.update_all(sheet)

42.5 ms ± 917 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


* Before optimisation : 

> SheetGeometry.update_all(sheet) : 83.4 ms

* After optimisation of the `_upcast` attribute:

> SheetGeometry.update_all(sheet) : 42.1 ms

* After writing the upcast in edge_df (not sure same computer)

> SheetGeometry.update_all(sheet) : 50.5 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

* With less calls to set_items:

> SheetGeometry.update_all(sheet) : 36.2 ms ± 238 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%snakeviz
Solver.find_energy_min(sheet, SheetGeometry, SheetModel)

 
*** Profile stats marshalled to file '/tmp/tmpuqit2fc4'. 


In [24]:
from functools import wraps
from tyssue import Epithelium

class CachedSheet(Sheet):
    
    def __init__(self, *args, **kwargs):
        self._cached_upcast = {}
        super().__init__(self, *args, **kwargs)
    
    def cacher(self, func, *args, **kwargs):
        @wraps(func)
        def cached_func(*args, **kwargs):
            h = hash((func, args[0]))
            if h in self._cached_upcast:
                print('from cache')
                return self._cached_upcast[h]
            else:
                print('set cache')
                res = func(*args, **kwargs)
                self._cached_upcast[h] = res
    
    @cacher
    def upcast_srce(self, *args, **kwargs):
        return super().upcast_srce(self, *args, **kwargs)
        




TypeError: cacher() missing 1 required positional argument: 'func'

In [21]:

sheet = CachedSheet('optim', load_datasets('data/hf5/before_apoptosis.hf5'),
                    config.geometry.cylindrical_sheet())

nd_specs = config.dynamics.quasistatic_sheet_spec()
d_specs = SheetModel.dimentionalize(nd_specs)
sheet.update_specs(d_specs)

NameError: name 'CachedSheet' is not defined